In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/413 Machine learning/Shakespeare
!git clone https://github.com/karpathy/nanoGPT.git


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/413 Machine learning/Shakespeare
Cloning into 'nanoGPT'...
remote: Enumerating objects: 682, done.
remote: Total 682 (delta 0), reused 0 (delta 0), pack-reused 682 (from 1)
Receiving objects: 100% (682/682), 952.47 KiB | 6.18 MiB/s, done.
Resolving deltas: 100% (385/385), done.


This above code is to load all the data and python file from the Github that provides an example of the nanoGPT trained to write Shakespeare's words.

In [ ]:
%cd nanoGPT
!ls

/content/drive/MyDrive/Colab Notebooks/413 Machine learning/Shakespeare/nanoGPT
assets	  config	   data     model.py   sample.py	   train.py
bench.py  configurator.py  LICENSE  README.md  scaling_laws.ipynb  transformer_sizing.ipynb


In [ ]:
!python data/shakespeare_char/prepare.py

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [ ]:
!python train.py config/train_shakespeare_char.py --device=cpu --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

It takes me 5minutes to train the model because I don't have a Nvidia GPU and eventually we get the loss function as 1.6982.  

In [ ]:
import tiktoken
!python sample.py --out_dir=out-shakespeare-char --device=cpu

Overriding: out_dir = out-shakespeare-char
Overriding: device = cpu
/content/drive/MyDrive/Colab Notebooks/413 Machine learning/Shakespeare/nanoGPT/sample.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues relat

Above is the lines I try to generate from the model I just trained. It imitated the way that Shakespeare write the scipt. I can see that sometimes it's not right because there are words like "succe" and "TLet" that don't really exist. The next step would be to eliminate such influence like words don't obey common grammar laws should be modified